## Imports

In [4]:
import numpy as np
from PIL import Image
import matplotlib
matplotlib.use('Agg') # Use non-interactive backend to prevent GUI issues.
import matplotlib.pyplot as plt
import time
import os
from tqdm import tqdm


## Image Processing and Dataset Creation 

In [17]:
class BorderDataset:
    """
    Loads and processes the 50x50 border image, converting it into a dataset
    of normalized coordinates and binary labels.
    """
    NETHERLANDS_ORANGE = np.array([255, 165, 0])
    BELGIUM_PURPLE = np.array([138, 43, 226])

    def __init__(self, image_path: str):
        """
        Initializes the dataset by loading and processing the image.
        """
        self.image_path = image_path
        self.binary_mask = self._create_binary_mask()
        self.height, self.width = self.binary_mask.shape
        self.pixels = self._prepare_pixels()
        print(f"Dataset created from '{self.image_path}' with {len(self.pixels)} points.")

    def _create_binary_mask(self) -> np.ndarray:
        """
        Opens the image and converts it to a 0-1 binary mask.
        """
        try:
            img = Image.open(self.image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Warning: Image file not found. Creating a dummy image at '{self.image_path}'.")
            img = self._create_dummy_image()
        
        img_array = np.array(img)
        dist_to_orange = np.linalg.norm(img_array - self.NETHERLANDS_ORANGE, axis=2)
        dist_to_purple = np.linalg.norm(img_array - self.BELGIUM_PURPLE, axis=2)
        mask = (dist_to_purple < dist_to_orange).astype(int)
        return mask

    def _create_dummy_image(self) -> Image:
        """
        Creates and saves a fallback 50x50 dummy image.
        """
        dummy_array = np.zeros((50, 50, 3), dtype=np.uint8)
        dummy_array[:, :] = self.NETHERLANDS_ORANGE
        dummy_array[10:25, 10:40] = self.BELGIUM_PURPLE
        dummy_array[30:45, 15:35] = self.BELGIUM_PURPLE
        img = Image.fromarray(dummy_array)
        
        directory = os.path.dirname(self.image_path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)
            
        img.save(self.image_path)
        return img

    def _prepare_pixels(self) -> list:
        """
        Generates a list of ((x, y), label) tuples for each pixel.
        """
        pixel_data = []
        for y in range(self.height):
            for x in range(self.width):
                normalized_x = x / (self.width - 1)
                normalized_y = y / (self.height - 1)
                label = self.binary_mask[y, x]
                pixel_data.append(((normalized_x, normalized_y), label))
        return pixel_data

    def get_shuffled_data(self) -> list:
        """
        Returns a randomly shuffled copy of the dataset.
        """
        shuffled_pixels = self.pixels.copy()
        np.random.shuffle(shuffled_pixels)
        return shuffled_pixels

class ReLU:
    """Rectified Linear Unit activation function."""
    def forward(self, x): return np.maximum(0, x)
    def backward(self, x): return (x > 0).astype(float)

class Sigmoid:
    """Sigmoid activation function."""
    def forward(self, x): return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    def backward(self, x):
        s = self.forward(x)
        return s * (1 - s)

class Linear:
    """A single fully-connected neural network layer."""
    def __init__(self, input_width, output_width, activation):
        self.weights = np.random.randn(input_width, output_width) * np.sqrt(2. / input_width)
        self.biases = np.zeros((1, output_width))
        self.activation = activation()
        self.input_data, self.z = None, None
        self.grad_weights = np.zeros_like(self.weights)
        self.grad_biases = np.zeros_like(self.biases)

    def forward(self, x):
        self.input_data = x
        self.z = np.dot(x, self.weights) + self.biases
        return self.activation.forward(self.z)

    def backward(self, grad_output):
        delta = grad_output * self.activation.backward(self.z)
        self.grad_weights += np.dot(self.input_data.T, delta)
        self.grad_biases += np.sum(delta, axis=0, keepdims=True)
        return np.dot(delta, self.weights.T)

class BCE:
    """Binary Cross-Entropy loss function."""
    def loss(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-12, 1 - 1e-12)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def backward(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-12, 1 - 1e-12)
        return (y_pred - y_true) / (y_pred * (1 - y_pred)) / y_true.size

class Model:
    """
    The main neural network model class.
    """
    def __init__(self, layers, loss_function='bce'):
        self.layers = layers
        self.loss_fn = BCE()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, grad_loss):
        for layer in reversed(self.layers):
            grad_loss = layer.backward(grad_loss)

    def train(self, x, y):
        y_pred = self.forward(x)
        loss = self.loss_fn.loss(y, y_pred)
        self.backward(self.loss_fn.backward(y, y_pred))
        return loss

    def predict(self, x):
        return self.forward(x)

    def zero_grad(self):
        for layer in self.layers:
            layer.grad_weights.fill(0)
            layer.grad_biases.fill(0)

    def update(self, learning_rate, grad_clip_value=1.0):
        for layer in self.layers:
            np.clip(layer.grad_weights, -grad_clip_value, grad_clip_value, out=layer.grad_weights)
            np.clip(layer.grad_biases, -grad_clip_value, grad_clip_value, out=layer.grad_biases)
            layer.weights -= learning_rate * layer.grad_weights
            layer.biases -= learning_rate * layer.grad_biases
        self.zero_grad()

    def save_to(self, path):
        params = {f'w_{i}': l.weights for i, l in enumerate(self.layers)}
        params.update({f'b_{i}': l.biases for i, l in enumerate(self.layers)})
        np.savez(path, **params)
        print(f"Model saved to {path}")

    def load_from(self, path):
        data = np.load(path)
        for i, layer in enumerate(self.layers):
            if layer.weights.shape != data[f'w_{i}'].shape or layer.biases.shape != data[f'b_{i}'].shape:
                raise ValueError(f"Architecture mismatch in layer {i}.")
            layer.weights, layer.biases = data[f'w_{i}'], data[f'b_{i}']
        print(f"Model loaded from {path}")

def positional_encoding(coords, num_frequencies):
    """
    Encodes coordinates to a higher dimension using sine and cosine functions.
    """
    frequencies = 2**np.linspace(0, num_frequencies-1, num_frequencies)
    encoded_coords = []
    for coord in coords.T: 
        sines = np.sin(2 * np.pi * coord[:, None] * frequencies)
        cosines = np.cos(2 * np.pi * coord[:, None] * frequencies)
        encoded_coords.extend([sines, cosines])
    return np.hstack(encoded_coords)

def plot_final_summary(history, model, dataset, run_folder, username, num_frequencies):
    """
    Generates and saves a single, consolidated plot.
    """
    print("Generating final summary plot...")
    plt.close('all') 
    
    h, w = dataset.height, dataset.width
    x_coords = np.linspace(0, 1, w)
    y_coords = np.linspace(0, 1, h)
    grid_x, grid_y = np.meshgrid(x_coords, y_coords)
    all_coords_raw = np.vstack([grid_x.ravel(), grid_y.ravel()]).T
    
    all_coords_encoded = positional_encoding(all_coords_raw, num_frequencies)
    
    predictions_raw = model.predict(all_coords_encoded)
    prediction_map = (predictions_raw > 0.5).astype(int).reshape((h, w))
    ground_truth_map = dataset.binary_mask
    error_map = np.abs(ground_truth_map - prediction_map)

    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(2, 3, height_ratios=[1, 1.2])
    fig.suptitle(f'Training and Prediction Summary - {username}', fontsize=20)

    ax_loss = fig.add_subplot(gs[0, 0])
    ax_loss.plot(history['loss'])
    ax_loss.set_title("Average Loss vs. Epochs")
    ax_loss.set_xlabel("Epoch"); ax_loss.set_ylabel("Loss"); ax_loss.grid(True)

    ax_acc = fig.add_subplot(gs[0, 1])
    ax_acc.plot(history['accuracy'])
    ax_acc.set_title("Accuracy vs. Epochs")
    ax_acc.set_xlabel("Epoch"); ax_acc.set_ylabel("Accuracy"); ax_acc.grid(True)
    
    ax_stats = fig.add_subplot(gs[0, 2])
    final_acc = history['accuracy'][-1]
    final_loss = history['loss'][-1]
    total_epochs = len(history['loss'])
    stats_text = (f"Final Accuracy: {final_acc:.4f}\n\n"
                  f"Final Loss: {final_loss:.4f}\n\n"
                  f"Total Epochs: {total_epochs}")
    ax_stats.text(0.5, 0.5, stats_text, ha='center', va='center', fontsize=14)
    ax_stats.axis('off')
    ax_stats.set_title("Final Stats")

    ax_gt = fig.add_subplot(gs[1, 0])
    ax_gt.imshow(ground_truth_map, cmap='viridis', interpolation='nearest')
    ax_gt.set_title("Ground Truth"); ax_gt.axis('off')

    ax_pred = fig.add_subplot(gs[1, 1])
    ax_pred.imshow(prediction_map, cmap='viridis', interpolation='nearest')
    ax_pred.set_title("Model Prediction"); ax_pred.axis('off')

    ax_err = fig.add_subplot(gs[1, 2])
    ax_err.imshow(error_map, cmap='hot', interpolation='nearest')
    ax_err.set_title("Error Map"); ax_err.axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    
    plot_path = os.path.join(run_folder, "final_summary.png")
    plt.savefig(plot_path)
    print(f"Plot saved to file: {plot_path}")
    
    plt.show() 
    plt.close(fig) 


def training_procedure(model, dataset, epochs, batch_size, initial_learning_rate, patience, username, num_frequencies, is_silent=False):
    """
    Handles the main training loop for all tasks.
    """
    data = dataset if isinstance(dataset, list) else dataset.get_shuffled_data()
    X_raw = np.array([item[0] for item in data])
    Y = np.array([item[1] for item in data]).reshape(-1, 1)
    
    X_encoded = positional_encoding(X_raw, num_frequencies)
    n_samples = len(data)
    
    history = {'loss': [], 'accuracy': []}
    
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    run_folder = f"runs/{timestamp}"
    os.makedirs(run_folder, exist_ok=True)
    if not is_silent:
        print(f"Saving results to {run_folder}")

    learning_rate = initial_learning_rate
    for epoch in range(epochs):
        if epoch > 0 and epoch % 100 == 0:
            learning_rate /= 2
            if not is_silent:
                print(f"\nLearning rate decayed to {learning_rate}\n")

        epoch_loss = 0
        permutation = np.random.permutation(n_samples)
        X_shuffled, Y_shuffled = X_encoded[permutation], Y[permutation]
        
        batch_iterator = range(0, n_samples, batch_size)
        if not is_silent:
            batch_iterator = tqdm(batch_iterator, desc=f"Epoch {epoch+1}/{epochs} (LR: {learning_rate})")

        for i in batch_iterator:
            batch_X = X_shuffled[i:i+batch_size]
            batch_Y = Y_shuffled[i:i+batch_size]
            loss = model.train(batch_X, batch_Y)
            model.update(learning_rate)
            epoch_loss += loss * batch_X.shape[0]
            if not is_silent:
                batch_iterator.set_postfix(loss=f"{loss:.4f}")
        
        avg_epoch_loss = epoch_loss / n_samples
        
        if np.isnan(avg_epoch_loss):
            print("\nCRITICAL ERROR: Loss has become NaN. Training cannot continue.")
            return None

        y_pred_full = model.predict(X_encoded)
        predictions = (y_pred_full > 0.5).astype(int)
        accuracy = np.mean(predictions == Y)
        
        history['loss'].append(avg_epoch_loss)
        history['accuracy'].append(accuracy)
        
        if not is_silent:
            if (epoch + 1) % 5 == 0 or epoch == epochs - 1:
                print(f"Epoch {epoch+1}/{epochs} | Avg Loss: {avg_epoch_loss:.4f} | Accuracy: {accuracy:.4f}")

        if epoch >= patience:
            if history['loss'][-1] >= (1.0 - 0.005) * history['loss'][-1 - patience]:
                if not is_silent:
                    print(f"Early stopping triggered at epoch {epoch+1}.")
                    if (epoch + 1) % 5 != 0:
                        print(f"Final Epoch {epoch+1}/{epochs} | Avg Loss: {avg_epoch_loss:.4f} | Accuracy: {accuracy:.4f}")
                break
    
    if not is_silent:
        print("\nTraining complete. Saving model...")
        model.save_to(os.path.join(run_folder, "final_model.npz"))
        print("Model saved. Now preparing final summary plot...")
        if not isinstance(dataset, list):
            plot_final_summary(history, model, dataset, run_folder, username, num_frequencies)

    return history

# This code will run when you execute the cell.
# It creates the border_dataset object in the main scope of the notebook,
# so it will be available for the next cell to use.

print("Executing Section 1.1: Dataset Creation")
image_file = "Dataset/border.png"
border_dataset = BorderDataset(image_file)

print("\nExecuting Section 1.2: Main Model Training")

num_frequencies = 10
input_dim = 4 * num_frequencies 

model = Model([
    Linear(input_dim, 256, ReLU),
    Linear(256, 256, ReLU),
    Linear(256, 1, Sigmoid)
])

print("Model created. Starting training on border map...")
# The history object is captured here to prevent the notebook from auto-printing it.
history = training_procedure(
    model=model,
    dataset=border_dataset,
    epochs=250, 
    batch_size=128,
    initial_learning_rate=0.01,
    patience=100,
    username="sudershan.sarraf",
    num_frequencies=num_frequencies,
    is_silent=False
)



Executing Section 1.1: Dataset Creation
Dataset created from 'Dataset/border.png' with 2500 points.

Executing Section 1.2: Main Model Training
Model created. Starting training on border map...
Saving results to runs/20251013-175009


Epoch 5/250 (LR: 0.01): 100%|██████████| 20/20 [00:00<00:00, 194.72it/s, loss=0.4201]


Epoch 5/250 | Avg Loss: 0.4446 | Accuracy: 0.8004


Epoch 10/250 (LR: 0.01): 100%|██████████| 20/20 [00:00<00:00, 215.67it/s, loss=0.3172]


Epoch 10/250 | Avg Loss: 0.3987 | Accuracy: 0.8164


...

Epoch 245/250 (LR: 0.0025): 100%|██████████| 20/20 [00:00<00:00, 198.73it/s, loss=0.2199]


Epoch 245/250 | Avg Loss: 0.1822 | Accuracy: 0.9404


Epoch 250/250 (LR: 0.0025): 100%|██████████| 20/20 [00:00<00:00, 202.07it/s, loss=0.1481]


Epoch 250/250 | Avg Loss: 0.1812 | Accuracy: 0.9416

Training complete. Saving model...
Model saved to runs/20251013-175009/final_model.npz
Model saved. Now preparing final summary plot...
Generating final summary plot...
Plot saved to file: runs/20251013-175009/final_summary.png


/tmp/ipykernel_1526/1232716160.py:244: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import time
import os
from tqdm import tqdm

# This script is fully self-contained and can be run in a new cell.
# It includes all necessary class and function definitions.

class BorderDataset:
    """
    Loads and processes the 50x50 border image, converting it into a dataset
    of normalized coordinates and binary labels.
    """
    NETHERLANDS_ORANGE = np.array([255, 165, 0])
    BELGIUM_PURPLE = np.array([138, 43, 226])

    def __init__(self, image_path: str):
        self.image_path = image_path
        self.binary_mask = self._create_binary_mask()
        self.height, self.width = self.binary_mask.shape
        self.pixels = self._prepare_pixels()
        print(f"Dataset created from '{self.image_path}' with {len(self.pixels)} points.")

    def _create_binary_mask(self) -> np.ndarray:
        try:
            img = Image.open(self.image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Warning: Image file not found. Creating a dummy image at '{self.image_path}'.")
            img = self._create_dummy_image()
        
        img_array = np.array(img)
        dist_to_orange = np.linalg.norm(img_array - self.NETHERLANDS_ORANGE, axis=2)
        dist_to_purple = np.linalg.norm(img_array - self.BELGIUM_PURPLE, axis=2)
        mask = (dist_to_purple < dist_to_orange).astype(int)
        return mask

    def _create_dummy_image(self) -> Image:
        dummy_array = np.zeros((50, 50, 3), dtype=np.uint8)
        dummy_array[:, :] = self.NETHERLANDS_ORANGE
        dummy_array[10:25, 10:40] = self.BELGIUM_PURPLE
        dummy_array[30:45, 15:35] = self.BELGIUM_PURPLE
        img = Image.fromarray(dummy_array)
        
        directory = os.path.dirname(self.image_path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)
            
        img.save(self.image_path)
        return img

    def _prepare_pixels(self) -> list:
        pixel_data = []
        for y in range(self.height):
            for x in range(self.width):
                normalized_x = x / (self.width - 1)
                normalized_y = y / (self.height - 1)
                label = self.binary_mask[y, x]
                pixel_data.append(((normalized_x, normalized_y), label))
        return pixel_data

    def get_shuffled_data(self) -> list:
        shuffled_pixels = self.pixels.copy()
        np.random.shuffle(shuffled_pixels)
        return shuffled_pixels

class ReLU:
    """Rectified Linear Unit activation function."""
    def forward(self, x): return np.maximum(0, x)
    def backward(self, x): return (x > 0).astype(float)

class Sigmoid:
    """Sigmoid activation function."""
    def forward(self, x): return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    def backward(self, x):
        s = self.forward(x)
        return s * (1 - s)

class Linear:
    """A single fully-connected neural network layer."""
    def __init__(self, input_width, output_width, activation):
        self.weights = np.random.randn(input_width, output_width) * np.sqrt(2. / input_width)
        self.biases = np.zeros((1, output_width))
        self.activation = activation()
        self.input_data, self.z = None, None
        self.grad_weights = np.zeros_like(self.weights)
        self.grad_biases = np.zeros_like(self.biases)

    def forward(self, x):
        self.input_data = x
        self.z = np.dot(x, self.weights) + self.biases
        return self.activation.forward(self.z)

    def backward(self, grad_output):
        delta = grad_output * self.activation.backward(self.z)
        self.grad_weights += np.dot(self.input_data.T, delta)
        self.grad_biases += np.sum(delta, axis=0, keepdims=True)
        return np.dot(delta, self.weights.T)

class BCE:
    """Binary Cross-Entropy loss function."""
    def loss(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-12, 1 - 1e-12)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def backward(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-12, 1 - 1e-12)
        return (y_pred - y_true) / (y_pred * (1 - y_pred)) / y_true.size

class Model:
    """The main neural network model class."""
    def __init__(self, layers, loss_function='bce'):
        self.layers = layers
        self.loss_fn = BCE()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, grad_loss):
        for layer in reversed(self.layers):
            grad_loss = layer.backward(grad_loss)

    def train(self, x, y):
        y_pred = self.forward(x)
        loss = self.loss_fn.loss(y, y_pred)
        self.backward(self.loss_fn.backward(y, y_pred))
        return loss

    def predict(self, x):
        return self.forward(x)

    def zero_grad(self):
        for layer in self.layers:
            layer.grad_weights.fill(0)
            layer.grad_biases.fill(0)

    def update(self, learning_rate, grad_clip_value=1.0):
        for layer in self.layers:
            np.clip(layer.grad_weights, -grad_clip_value, grad_clip_value, out=layer.grad_weights)
            np.clip(layer.grad_biases, -grad_clip_value, grad_clip_value, out=layer.grad_biases)
            layer.weights -= learning_rate * layer.grad_weights
            layer.biases -= learning_rate * layer.grad_biases
        self.zero_grad()

def positional_encoding(coords, num_frequencies):
    """Encodes coordinates to a higher dimension using sine and cosine functions."""
    frequencies = 2**np.linspace(0, num_frequencies-1, num_frequencies)
    encoded_coords = []
    for coord in coords.T: 
        sines = np.sin(2 * np.pi * coord[:, None] * frequencies)
        cosines = np.cos(2 * np.pi * coord[:, None] * frequencies)
        encoded_coords.extend([sines, cosines])
    return np.hstack(encoded_coords)

def training_procedure(model, dataset, epochs, batch_size, initial_learning_rate, patience, username, num_frequencies, is_silent=False):
    """Handles the main training loop for all tasks."""
    data = dataset if isinstance(dataset, list) else dataset.get_shuffled_data()
    X_raw = np.array([item[0] for item in data])
    Y = np.array([item[1] for item in data]).reshape(-1, 1)
    
    X_encoded = positional_encoding(X_raw, num_frequencies)
    n_samples = len(data)
    
    history = {'loss': [], 'accuracy': []}
    
    for epoch in range(epochs):
        epoch_loss = 0
        permutation = np.random.permutation(n_samples)
        X_shuffled, Y_shuffled = X_encoded[permutation], Y[permutation]
        
        for i in range(0, n_samples, batch_size):
            batch_X = X_shuffled[i:i+batch_size]
            batch_Y = Y_shuffled[i:i+batch_size]
            loss = model.train(batch_X, batch_Y)
            model.update(initial_learning_rate)
            epoch_loss += loss * batch_X.shape[0]
        
        avg_epoch_loss = epoch_loss / n_samples
        
        if np.isnan(avg_epoch_loss):
            print("\nCRITICAL ERROR: Loss has become NaN.")
            return None

        y_pred_full = model.predict(X_encoded)
        predictions = (y_pred_full > 0.5).astype(int)
        accuracy = np.mean(predictions == Y)
        
        history['loss'].append(avg_epoch_loss)
        history['accuracy'].append(accuracy)

        if epoch >= patience:
            if history['loss'][-1] >= (1.0 - 0.005) * history['loss'][-1 - patience]:
                if not is_silent:
                    print(f"Early stopping triggered at epoch {epoch+1}.")
                break
    return history

def xor_training_procedure(model, dataset, epochs, batch_size, learning_rate, patience):
    """A simplified training loop for the XOR test without positional encoding."""
    X = np.array([item[0] for item in dataset])
    Y = np.array([item[1] for item in dataset]).reshape(-1, 1)
    n_samples = len(dataset)
    
    history = {'loss': [], 'accuracy': []}
    
    for epoch in range(epochs):
        epoch_loss = 0
        permutation = np.random.permutation(n_samples)
        X_shuffled, Y_shuffled = X[permutation], Y[permutation]
        
        for i in range(0, n_samples, batch_size):
            batch_X = X_shuffled[i:i+batch_size]
            batch_Y = Y_shuffled[i:i+batch_size]
            loss = model.train(batch_X, batch_Y)
            model.update(learning_rate)
            epoch_loss += loss * batch_X.shape[0]
        
        avg_epoch_loss = epoch_loss / n_samples
        
        if np.isnan(avg_epoch_loss):
            return None

        y_pred_full = model.predict(X)
        predictions = (y_pred_full > 0.5).astype(int)
        accuracy = np.mean(predictions == Y)
        
        history['loss'].append(avg_epoch_loss)
        history['accuracy'].append(accuracy)

        if accuracy == 1.0 and epoch > 100: # Stop once 100% is reached
            break
            
    return history

# Sanity Check: XOR Test
def execute_xor_test():
    """Performs the XOR sanity check to verify the MLP implementation."""
    print("\nExecuting Sanity Check: The XOR Problem")
    xor_data = [((0, 0), 0), ((0, 1), 1), ((1, 0), 1), ((1, 1), 0)]
    
    # The model for XOR takes 2 raw inputs, not encoded ones.
    model = Model([
        Linear(2, 8, ReLU),
        Linear(8, 1, Sigmoid)
    ])
    
    print("Training model on XOR data...")
    # Use the dedicated XOR training loop
    history = xor_training_procedure(
        model=model, dataset=xor_data, epochs=3000, batch_size=4,
        learning_rate=0.1, patience=200
    )
    
    if history:
        final_accuracy = history['accuracy'][-1]
        print(f"XOR test final accuracy: {final_accuracy:.4f}")
        if final_accuracy == 1.0:
            print("XOR Test PASSED: Model achieved 100% accuracy.")
        else:
            print("XOR Test FAILED: Model did not achieve 100% accuracy.")

# Sanity Check: Gradient Check
def execute_gradient_check():
    """Performs a gradient check by comparing analytical and numerical gradients."""
    print("\nExecuting Sanity Check: Gradient Check")
    
    x = np.array([[0.5, 0.5]])
    y = np.array([[1]])
    num_frequencies = 4
    input_dim = 4 * num_frequencies
    x_encoded = positional_encoding(x, num_frequencies)

    model = Model([Linear(input_dim, 2, ReLU), Linear(2, 1, Sigmoid)])
    
    model.train(x_encoded, y)
    analytical_grads = [np.copy(l.grad_weights) for l in model.layers]
    model.zero_grad()

    epsilon = 1e-5
    numerical_grads = [np.zeros_like(l.weights) for l in model.layers]
    
    for l_idx, layer in enumerate(model.layers):
        for i in range(layer.weights.shape[0]):
            for j in range(layer.weights.shape[1]):
                original_weight = layer.weights[i, j]
                
                layer.weights[i, j] = original_weight + epsilon
                loss_plus = model.loss_fn.loss(y, model.forward(x_encoded))
                
                layer.weights[i, j] = original_weight - epsilon
                loss_minus = model.loss_fn.loss(y, model.forward(x_encoded))
                
                layer.weights[i, j] = original_weight
                
                numerical_grads[l_idx][i, j] = (loss_plus - loss_minus) / (2 * epsilon)

    total_relative_error = 0
    print("Comparing analytical and numerical gradients...")
    for l_idx in range(len(model.layers)):
        numerator = np.linalg.norm(analytical_grads[l_idx] - numerical_grads[l_idx])
        denominator = np.linalg.norm(analytical_grads[l_idx]) + np.linalg.norm(numerical_grads[l_idx])
        relative_error = numerator / denominator if denominator > 1e-8 else 0
        total_relative_error += relative_error
        print(f"Layer {l_idx} Relative Error: {relative_error}")

    if total_relative_error < 1e-6:
        print("Gradient Check PASSED.")
    else:
        print("Gradient Check FAILED.")

# Map Prediction and Analysis
def execute_analysis(dataset):
    """Trains models with varying architectures and plots the results."""
    print("\nExecuting Map Prediction and Analysis")
    num_frequencies = 10
    input_dim = 4 * num_frequencies

    print("\nAnalyzing effect of network depth...")
    depths = [2, 3, 4, 5]
    width = 128
    depth_accuracies = []
    for depth in depths:
        print(f"Training model with depth {depth} and width {width}...")
        layers = [Linear(input_dim, width, ReLU)]
        for _ in range(depth - 1):
            layers.append(Linear(width, width, ReLU))
        layers.append(Linear(width, 1, Sigmoid))
        model = Model(layers)
        
        history = training_procedure(model, dataset, epochs=150, batch_size=128, initial_learning_rate=0.01, patience=20, username=f"Depth_{depth}", num_frequencies=num_frequencies, is_silent=True)
        if history:
            final_accuracy = history['accuracy'][-1]
            depth_accuracies.append(final_accuracy)
            print(f"Final accuracy for depth {depth}: {final_accuracy:.4f}")

    print("\nAnalyzing effect of network width...")
    widths = [32, 64, 128, 256]
    depth = 3
    width_accuracies = []
    for width in widths:
        print(f"Training model with depth {depth} and width {width}...")
        layers = [Linear(input_dim, width, ReLU)]
        for _ in range(depth - 1):
            layers.append(Linear(width, width, ReLU))
        layers.append(Linear(width, 1, Sigmoid))
        model = Model(layers)

        history = training_procedure(model, dataset, epochs=150, batch_size=128, initial_learning_rate=0.01, patience=20, username=f"Width_{width}", num_frequencies=num_frequencies, is_silent=True)
        if history:
            final_accuracy = history['accuracy'][-1]
            width_accuracies.append(final_accuracy)
            print(f"Final accuracy for width {width}: {final_accuracy:.4f}")

    plt.close('all')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Architecture Analysis')
    ax1.plot(depths, depth_accuracies, marker='o')
    ax1.set_title("Accuracy vs. Depth (Width=128)")
    ax1.set_xlabel("Number of Hidden Layers"); ax1.set_ylabel("Final Accuracy"); ax1.grid(True)
    
    ax2.plot(widths, width_accuracies, marker='o')
    ax2.set_title("Accuracy vs. Width (Depth=3)")
    ax2.set_xlabel("Neurons per Hidden Layer"); ax2.set_ylabel("Final Accuracy"); ax2.grid(True)
    
    analysis_plot_path = 'runs/architecture_analysis.png'
    os.makedirs('runs', exist_ok=True)
    plt.savefig(analysis_plot_path)
    print(f"Analysis plot saved to {analysis_plot_path}")
    plt.show()

# Final Challenge
def execute_challenge(dataset):
    """Attempts to solve the final challenge goals."""
    print("\nExecuting Final Challenge")
    num_frequencies = 10
    input_dim = 4 * num_frequencies
    
    print("\nGoal 1: Minimize model size for >91% accuracy.")
    small_model = Model([
        Linear(input_dim, 64, ReLU),
        Linear(64, 64, ReLU),
        Linear(64, 1, Sigmoid)
    ])
    training_procedure(small_model, dataset, epochs=250, batch_size=128, initial_learning_rate=0.01, patience=30, username="SmallModel_Challenge", num_frequencies=num_frequencies)

    print("\nGoal 2: Minimize training samples for >91% accuracy.")
    fast_model = Model([
        Linear(input_dim, 256, ReLU),
        Linear(256, 256, ReLU),
        Linear(256, 1, Sigmoid)
    ])
    training_procedure(fast_model, dataset, epochs=100, batch_size=256, initial_learning_rate=0.02, patience=20, username="FastConverge_Challenge", num_frequencies=num_frequencies)

def main_additions():
    """Main function to execute the additional sections of Q1."""
    RUN_SANITY_CHECKS = True
    RUN_ANALYSIS = False
    RUN_CHALLENGE = False

    if RUN_SANITY_CHECKS:
        execute_xor_test()
        execute_gradient_check()

    # Create the dataset object only if needed for analysis or challenge
    if RUN_ANALYSIS or RUN_CHALLENGE:
        print("Creating border_dataset object for analysis/challenge...")
        border_dataset = BorderDataset(image_path="Dataset/border.png")
        if RUN_ANALYSIS:
            execute_analysis(border_dataset)

        if RUN_CHALLENGE:
            execute_challenge(border_dataset)

# This call will now run the functions in this cell.
main_additions()




Executing Sanity Check: The XOR Problem
Training model on XOR data...
XOR test final accuracy: 1.0000
XOR Test PASSED: Model achieved 100% accuracy.

Executing Sanity Check: Gradient Check
Comparing analytical and numerical gradients...
Layer 0 Relative Error: 2.0770530088677715e-11
Layer 1 Relative Error: 1.1733377223073913e-11
Gradient Check PASSED.
